In [ ]:
from  functools import partial
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate

plt.rcParams['figure.max_open_warning'] = 100
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['mathtext.bf'] = 'Arial'
plt.rcParams['axes.formatter.use_mathtext'] = True
    
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif': ['Arial']})
rc('text', usetex=False)

def set_font(p,plt):
    for size in ['axes.titlesize','axes.labelsize','font.size','figure.titlesize','legend.fontsize','xtick.labelsize','ytick.labelsize']:
        plt.rcParams[size] = p

set_font(26,plt)

In [ ]:
import pandas as pd
import numpy as np
import json

data = json.load(open("./raw_data.json",'r'))
state_wise = {}
for i in data['raw_data']:
    try:
        state_wise[i['detectedstate']]['raw'] += [i]
    except:
        state_wise[i['detectedstate']] = {}
        state_wise[i['detectedstate']]['raw'] = []
        state_wise[i['detectedstate']]['raw'] += [i]
        
dates = ['30/01/2020','31/01/2020']
for i in range(1,30):
    dates += ["{}/02/2020".format(str(100+i)[-2:])]
for i in range(1,32):
    dates += ["{}/03/2020".format(str(100+i)[-2:])]
for i in range(1,6):
    dates += ["{}/04/2020".format(str(100+i)[-2:])]

def add_data(state):
    state['data'] = {}
    for i in dates:
        state['data'][i] = [0,0,0]
    for p in state['raw']:
        date_a = p['dateannounced']
        date_c = p['statuschangedate']
        state['data'][date_a][0] += 1
        if p['currentstatus']=="Recovered":
            state['data'][date_c][1] += 1 
        elif p['currentstatus']=="Deceased":
            state['data'][date_c][2] += 1 
    return state

new_data = {}
for i in state_wise:
    if i=='':
        pass
    else:
        new_data[i] = add_data(state_wise[i])
   
all_dfs = None
for i in new_data:
    df = pd.DataFrame()
    df['Date'] = [j for j in new_data[i]['data'].keys()]
    df[i+'_I'] = np.array([j for j in new_data[i]['data'].values()])[:,0]
    df[i+'_R'] = np.array([j for j in new_data[i]['data'].values()])[:,1]
    df[i+'_D'] = np.array([j for j in new_data[i]['data'].values()])[:,2]
    try:
        if all_dfs==None:
            all_dfs = df
    except:
        all_dfs = pd.merge(all_dfs,df,how='outer',on='Date')
        
        
new_dfs = all_dfs[all_dfs.columns[1:]].cumsum()
new_dfs.columns = [i+'_cumsum' for i in all_dfs.columns[1:]]
new_dfs['Date'] = all_dfs['Date']
big_df = pd.merge(all_dfs,new_dfs,how='outer',on='Date')

big_df['India_I'] = big_df[[i for i in big_df.columns if 'I_cumsum' in i]].sum(axis=1)
big_df['India_R'] = big_df[[i for i in big_df.columns if 'R_cumsum' in i]].sum(axis=1)
big_df['India_D'] = big_df[[i for i in big_df.columns if 'D_cumsum' in i]].sum(axis=1)

big_df.columns = [i.replace(' ','') for i in big_df.columns]

big_df.to_csv('./BIG_DF.csv',index=False)
# 15/03/2020

In [ ]:
statewise_pop = {'UttarPradesh': 199812341,
 'Maharashtra': 112374333,
 'Bihar': 104099452,
 'WestBengal': 91276115,
 'MadhyaPradesh': 72626809,
 'TamilNadu': 72147030,
 'Rajasthan': 68548437,
 'Karnataka': 61095297,
 'Gujarat': 60439692,
 'AndhraPradesh': 49577103,
 'Odisha': 41974218,
 'Telangana': 35003674,
 'Kerala': 33406061,
 'Jharkhand': 32988134,
 'Assam': 31205576,
 'Punjab': 27743338,
 'Chhattisgarh': 25545198,
 'Haryana': 25351462,
 'Uttarakhand': 10086292,
 'HimachalPradesh': 6864602,
 #'Tripura': 3673917,
 #'Meghalaya': 2966889,
 'Manipur': 2570390,
 #'Nagaland': 1978502,
 'Goa': 1458545,
 'ArunachalPradesh': 1383727,
 'Mizoram': 1097206,
 #'Sikkim': 610577,
 'Delhi': 16787941,
 'JammuandKashmir': 12267032,
 'Puducherry': 1247953,
 'Chandigarh': 1055450,
 #'DadraandNagarHaveliandDamanandDiu': 585764,
 'AndamanandNicobarIslands': 380581,
 'Ladakh': 274000,
 #'Lakshadweep': 64473,
                }

In [ ]:
class Multi_SEIR():
    def __init__(self,sub_population_num,l1=None):
        self.n_population = sub_population_num
        self.y = None
        if type(l1)==type([]):
            X0,W0,Y0,Z0,D0 = l1
        else:
            X0 = np.ones((sub_population_num,sub_population_num))*1000
            W0 = np.zeros((sub_population_num,sub_population_num))
            Y0 = np.ones((sub_population_num,sub_population_num))*1
            Z0 = np.zeros((sub_population_num,sub_population_num))
            D0 = np.zeros((sub_population_num,sub_population_num))
        y0 = np.hstack([X0.ravel(),W0.ravel(),Y0.ravel(),Z0.ravel(),D0.ravel()])
        self.y0 = y0
        self.diff = {'X_d':[], 'W_d':[], 'Y_d':[], 'Z_d':[], 'D_d':[],}
        self._spacer = '\n'
    
    def sumit(self,t,y,fs,*args,**kwargs):
        s = np.zeros(self.n_population*self.n_population)
        for i in fs:
            s += i(t,y,self.y,**kwargs)
        return s
    
    def solve(self,t,**kwargs):
        def system(y,t):
            ret = np.hstack([self.sumit(t,y,self.diff[k]) for k in self.diff])
            return ret.ravel()
        args = ()
        self.y = scipy.integrate.odeint(system,self.y0,t,args=args,**kwargs)
    
    def __repr_text(self,text):
        return (self._spacer+text)
    
    def __repr__(self):
        t = self.__repr_text
        text = t("Multi-SEIR model with following parameters:")
        text += t("Total number of sub population: {}".format(self.n_population))
        self._spacer = "\n"
        text += t("Initial conditions: ")
        
        self._spacer = "\n\t"
        n = self.n_population*self.n_population
        names = list(self.diff.keys())
        for i in range(5):
            text += t("{}: ".format(names[i])+self.y0[i*n:i*n+n].reshape(self.n_population,self.n_population).__repr__())

        self._spacer = "\n\n"
        text += t("Transitions:")
        for k,v in self.diff.items():
            self._spacer = "\n\t\n"
            text += t("{}: ".format(k))
            self._spacer = "\n\t\t"
            text += t("{}".format(v))
        
        if type(self.y)!=type(None):
            for i in range(5):
                text += t("{}: ".format(names[i])+self.y[-1][i*n:i*n+n].reshape(self.n_population,self.n_population).__repr__())
            
        return text
    
    def add(self,name,fs):
        if type(fs)!=type([]):
            fs = [fs]
        for f in fs:
            self.diff[name] += [partial(f,dim=self.n_population)]
            

## Initial condition

In [ ]:
l2 = list(statewise_pop.keys())
l2.sort()

l1 = [i[:-9] for i in big_df.columns if 'I_cumsum' in i]
l1.sort()
# for i,j in zip(l1,l2):
#     print(i,'==',j,'=>',i==j)

In [ ]:
import pickle

with open('./R0.dict','rb') as file:
    R0_dict = pickle.load(file)

R0 = [ R0_dict[l2[i]] for i in range(len(l2))]


In [ ]:
dim = len(l2)
X0 = np.zeros((dim,dim))
W0 = np.zeros((dim,dim))
Y0 = np.zeros((dim,dim))
Z0 = np.zeros((dim,dim))
D0 = np.zeros((dim,dim))

In [ ]:
capitals_dict = {'AndhraPradesh': 'Amaravati',
 'ArunachalPradesh': 'Itanagar',
 'Assam': 'Dispur',
 'Bihar': 'Patna',
 'Chhattisgarh': 'Naya Raipur',
 'Goa': 'Panaji',
 'Gujarat': 'Gandhinagar',
 'Haryana': 'Chandigarh',
 'HimachalPradesh': 'Shimla',
 'Jharkhand': 'Ranchi',
 'Karnataka': 'Bangalore',
 'Kerala': 'Thiruvananthapuram',
 'MadhyaPradesh': 'Bhopal',
 'Maharashtra': 'Mumbai',
 'Manipur': 'Imphal',
 'Meghalaya': 'Shillong',
 'Mizoram': 'Aizawl',
 'Nagaland': 'Kohima',
 'Odisha': 'Bhubaneswar',
 'Punjab': 'Chandigarh',
 'Rajasthan': 'Jaipur',
 'Sikkim': 'Gangtok',
 'TamilNadu': 'Chennai',
 'Telangana': 'Hyderabad',
 'Tripura': 'Agartala',
 'UttarPradesh': 'Lucknow',
 'Uttarakhand': 'Dehradun',
 'WestBengal': 'Kolkata',
 'AndamanandNicobarIslands': 'Port Blair',
 'Chandigarh': 'Chandigarh',
 'Delhi': 'Delhi',
 'JammuandKashmir': 'Jammu',
 'Ladakh': 'Kargil',
 'Puducherry': 'Puducherry',                
}
lat_long_dict = {'Nanjangud': [12.12, 76.68],
 'Chittorgarh': [24.879999, 74.629997],
 'Ratnagiri': [16.994444, 73.300003],
 'Goregaon': [19.155001, 72.849998],
 'Pindwara': [24.7945, 73.055],
 'Raipur': [21.25, 81.629997],
 'Gokak': [16.1667, 74.833298],
 'Lucknow': [26.85, 80.949997],
 'Delhi': [28.610001, 77.230003],
 'Mumbai': [19.07609, 72.877426],
 'Sagar': [23.83403, 78.746567],
 'Jalpaiguri': [26.540457, 88.719391],
 'Pakur': [24.633568, 87.849251],
 'Sardarshahar': [28.440554, 74.493011],
 'Sirohi': [24.882618, 72.858894],
 'Jaysingpur': [16.779877, 74.556374],
 'Ramanagara': [12.715035, 77.281296],
 'Chikkaballapura': [13.435455, 77.731476],
 'Channapatna': [12.651805, 77.208946],
 'Surendranagar': [22.728392, 71.637077],
 'Thiruvalla': [9.383452, 76.574059],
 'Ranebennur': [14.623801, 75.621788],
 'Karaikal': [10.92544, 79.838005],
 'Belgaum': [15.852792, 74.498703],
 'Chatrapur': [19.354979, 84.986732],
 'Suri': [23.905445, 87.52462],
 'Bhubaneswar': [20.296059, 85.824539],
 'Mahuva': [21.092159, 71.770462],
 'Jagadhri': [30.172716, 77.299492],
 'Barh': [25.477585, 85.709091],
 'Bhusawal': [21.045521, 75.801094],
 'Alipurduar': [26.49189, 89.5271],
 'Kollam': [8.893212, 76.614143],
 'Medinipur': [22.430889, 87.321487],
 'Patan': [23.849325, 72.126625],
 'Mettur': [11.786253, 77.800781],
 'Huliyar': [13.583274, 76.540154],
 'Harihar': [14.530457, 75.801094],
 'Rasayani': [18.901457, 73.176132],
 'Haringhata': [22.96051, 88.567406],
 'Kushtagi': [15.756595, 76.192696],
 'Jadugora': [22.656015, 86.352882],
 'Orai': [25.98, 79.470001],
 'Surajpur': [23.223047, 82.87056],
 'Ambernath': [19.186354, 73.191948],
 'Malerkotla': [30.525005, 75.890121],
 'Jorapokhar': [22.422455, 85.760651],
 'Vizianagaram': [18.106659, 83.395554],
 'Durg': [21.190449, 81.28492],
 'Himmatnagar': [23.597969, 72.969818],
 'Sambhal': [28.590361, 78.571762],
 'Harnaut': [25.369179, 85.53006],
 'Port Blair': [11.623377, 92.726486],
 'Suti': [24.618393, 88.024338],
 'Banswara': [23.546757, 74.43383],
 'Batumi': [41.643414, 41.6399],
 'Manikchak': [25.077787, 87.900375],
 'Roorkee': [29.854263, 77.888],
 'Kavali': [14.913181, 79.992981],
 'Dharmavaram': [14.413745, 77.712616],
 'Siddipet': [18.101904, 78.852074],
 'Dhanpuri': [23.173939, 81.565125],
 'Chirala': [15.812074, 80.355377],
 'Markapur': [15.764501, 79.259491],
 'Chalakudy': [10.311879, 76.331978],
 'Gondal': [21.961946, 70.792297],
 'Bhimavaram': [16.544893, 81.52124],
 'Jalgaon Jamod': [21.04954, 76.532028],
 'Paltan Bazaar': [26.182245, 91.754723],
 'Hodal': [27.897551, 77.384117],
 'Ausa': [18.245655, 76.505356],
 'Mahidpur': [23.486839, 75.659157],
 'Gurdaspur': [32.041943, 75.405334],
 'Domchanch': [24.47438, 85.688744],
 'Barjora': [23.427221, 87.287018],
 'Saint Dnyaneshwar Garden and Paithan park': [19.487707, 75.380768],
 'Sinnar': [19.85306, 74.000633],
 'Guntakal': [15.167409, 77.373627],
 'Lalgola': [24.417534, 88.250343],
 'Hoshangabad': [22.744108, 77.736969],
 'Proddatur': [14.752805, 78.552757],
 'RL Infotechh & Solutions': [23.520399, 87.311897],
 'Pali': [25.771315, 73.323685],
 'Palwal': [28.148735, 77.332024],
 'Gohana': [29.138407, 76.693245],
 'Munger': [25.37571, 86.474373],
 'Yavatmal': [20.388794, 78.120407],
 'Bokaro Steel City': [23.669296, 86.151115],
 'Jetpur': [21.761524, 70.627625],
 'Basirhat': [22.657402, 88.86718],
 'Konnagar': [22.700474, 88.319069],
 'Ranchi': [23.344101, 85.309563],
 'Gudur': [14.146319, 79.850388],
 'Gola Gokarannath': [28.078636, 80.471588],
 'Shikohabad': [27.108416, 78.584602],
 'Tirumangalam': [9.823619, 77.986565],
 'Anakaputhur': [12.946366, 79.959244],
 'Suryapet': [17.143908, 79.623924],
 'Udupi': [13.340881, 74.742142],
 'Begusarai': [25.416676, 86.129379],
 'Contai': [21.779188, 87.744629],
 'Naya Raipur': [21.164993, 81.775307],
 'Payradanga': [23.125587, 88.546867],
 'Maheshtala': [22.511976, 88.250992],
 'Vaniyambadi': [12.695032, 78.621887],
 'Shahdol': [23.302189, 81.356804],
 'Aruppukkottai': [9.515625, 78.100014],
 'Salipur': [20.484348, 86.119194],
 'Sasaram': [24.953514, 84.011787],
 'Bihta': [25.563322, 84.869804],
 'Dahod': [22.835602, 74.255989],
 'Najibabad': [29.607981, 78.342674],
 'Bagaha': [27.122196, 84.072235],
 'Jamalpur': [23.061089, 87.992584],
 'Fatehpur': [27.99502, 74.961792],
 'Dindigul': [10.365581, 77.970657],
 'Chakradharpur': [22.668312, 85.625511],
 'Porbandar': [21.640575, 69.605965],
 'Ghatal': [22.663696, 87.746803],
 'Khidirpur': [22.542061, 88.318954],
 'Mandya': [12.523731, 76.894684],
 'Dahanu': [19.990751, 72.743675],
 'Sri Muktsar Sahib': [30.480042, 74.518204],
 'Kanchipuram': [12.834174, 79.703644],
 'Dhanpur': [22.633492, 74.095993],
 'Bengaluru': [12.971599, 77.594566],
 'Muzaffarnagar': [29.471397, 77.696732],
 'Chhatarpur': [24.916355, 79.581184],
 'Bongaigaon': [26.500999, 90.535233],
 'RAK Tiles': [31.604889, 76.919533],
 'Nilanga': [18.125875, 76.750969],
 'Amreli': [21.603176, 71.222084],
 'Bulandshahar': [28.411331, 77.848434],
 'Kharagpur': [22.34601, 87.231972],
 'Arrah': [25.556044, 84.660332],
 'Seoni': [22.086868, 79.543488],
 'Siliguri': [26.732311, 88.410286],
 'Mangaluru': [12.915605, 74.855965],
 'Kandi': [23.9468, 88.049713],
 'Khammam': [17.247253, 80.151443],
 'Morshi': [21.324196, 78.013832],
 'Vijaypur': [32.564522, 75.023148],
 'Santha': [26.994154, 83.034798],
 'Chalisgaon': [20.464079, 74.996857],
 'Etawah': [26.811712, 79.004692],
 'Simlapal': [22.916706, 87.070351],
 'Lakhimpur': [27.94908, 80.782402],
 'Masaurhi': [25.35128, 85.031143],
 'Bhatta': [23.00795, 72.553757],
 'Sultanpur': [26.264776, 82.072708],
 'Kotulpur': [23.012794, 87.593948],
 'Pithampur': [22.610683, 75.680862],
 'Bela Pratapgarh': [25.912968, 81.991653],
 'Ghazipur': [25.584042, 83.577019],
 'Buxar': [25.564711, 83.977745],
 'Baran': [25.101145, 76.513161],
 'Saideep Enterprises': [19.969227, 73.743195],
 'Hazaribagh': [23.99662, 85.36911],
 'Rudrapur': [28.978298, 79.399612],
 'Success Point': [20.112093, 79.118698],
 'Balaghat': [21.812876, 80.18383],
 'Thodupuzha': [9.897157, 76.713409],
 'Kothagudem': [17.554356, 80.619736],
 'Debra': [22.369043, 87.55442],
 'Khambhat': [22.318083, 72.618988],
 'Kalaburagi': [17.320486, 76.839752],
 'Palampur': [32.110542, 76.536224],
 'Bargarh': [21.342585, 83.624199],
 'Jaunpur': [25.748695, 82.698441],
 'Bahraich': [27.570839, 81.598022],
 'Shegaon': [20.792713, 76.694328],
 'Ballari': [15.139393, 76.92144],
 'Hapur': [28.730579, 77.775879],
 'Kashipur': [29.210421, 78.96183],
 'Sikandra': [26.367811, 79.629158],
 'Palakkad': [10.784703, 76.653145],
 'Almora': [29.594189, 79.653893],
 'Dhar': [22.601292, 75.302467],
 'Ameerpet': [17.437462, 78.448288],
 'Georai': [19.263334, 75.752426],
 'Kodad': [16.99708, 79.967361],
 'Kalaigaon': [26.57448, 91.975243],
 'Newasa': [19.551146, 74.928162],
 'Jalgaon': [21.004194, 75.563942],
 'Mahasamund': [21.109133, 82.0979],
 'Giridih': [24.184713, 86.302193],
 'Purnea': [25.771103, 87.482185],
 'Kotputli': [27.703669, 76.201195],
 'Jhalawar': [24.597349, 76.16098],
 'Sitarganj': [28.923548, 79.700722],
 'Barauda': [21.191774, 81.718285],
 'Sangareddy': [17.611629, 78.08181],
 'Azamgarh': [26.072067, 83.185654],
 'Suratgarh': [29.320511, 73.899399],
 'Gingee': [12.252576, 79.416924],
 'Samastipur': [25.862968, 85.781029],
 'Jahanabad': [25.213928, 84.989555],
 'Kekri': [25.973787, 75.151535],
 'Dholpur': [26.702518, 77.893394],
 'Hosur': [12.735126, 77.829338],
 'Bisalpur': [28.293783, 79.804459],
 'Jhansi': [25.436298, 78.567352],
 'Korba': [22.363848, 82.73484],
 'Tadepalligudem': [16.814524, 81.526558],
 'Rajnandgaon': [21.097855, 81.033707],
 'Gumla': [23.041924, 84.540833],
 'Kuchaman City': [27.147869, 74.859489],
 'Sankari': [11.477696, 77.873886],
 'Forbesganj': [26.298552, 87.265388],
 'Thandalam': [12.999464, 80.117729],
 'Etah': [27.55875, 78.662567],
 'Nabadwip': [23.406979, 88.365593],
 'Sri Ganganagar': [29.903839, 73.87719],
 'Hanamkonda': [18.007202, 79.558296],
 'Ramagundam': [18.76182, 79.480904],
 'Mau': [25.949638, 83.558647],
 'Gangakhed': [18.966354, 76.748077],
 'Rahon': [31.052128, 76.1175],
 'Mansa': [29.988245, 75.379257],
 'Narwana': [29.592957, 76.119247],
 'Pulivendula': [14.422232, 78.226341],
 'Brahmapur': [19.314962, 84.79409],
 'Kadayanallur': [9.077854, 77.345184],
 'Bathinda': [30.210995, 74.945473],
 'Botad': [22.170424, 71.668427],
 'Darjeeling': [27.036007, 88.262672],
 'Chiplun': [17.53231, 73.517792],
 'Haldwani': [29.219677, 79.512459],
 'Cuddalore': [11.744699, 79.768021],
 'Bharuch': [21.705723, 72.998199],
 'Bhadohi': [25.38727, 82.568031],
 'Vaijapur': [19.925682, 74.728477],
 'Vadakara': [11.608495, 75.591705],
 'Sitamarhi': [26.587299, 85.501183],
 'Rajgir': [25.017258, 85.416161],
 'Shimoga': [13.92993, 75.5681],
 'Unnao': [26.536938, 80.48996],
 'Rangia': [26.437252, 91.62011],
 'Chandrapur': [19.970324, 79.30336],
 'Silchar': [24.833946, 92.779282],
 'Hailakandi': [24.68469, 92.564339],
 'Aizawl': [23.727106, 92.717636],
 'South Dum Dum': [22.607672, 88.394493],
 'Kavathe Mahankal': [17.006298, 74.865372],
 'Davanagere': [14.464103, 75.921661],
 'Mhow': [22.552437, 75.756531],
 'Moga': [30.81646, 75.171707],
 'Bhatpara': [22.859726, 88.404839],
 'Batala': [31.823462, 75.205063],
 'Farrukhabad': [27.387049, 79.588127],
 'Nohar': [29.185776, 74.771782],
 'Jharia': [23.742628, 86.41114],
 'Hoshiarpur': [31.527355, 75.913673],
 'Balurghat': [25.237284, 88.783058],
 'Madhubani': [26.347883, 86.071861],
 'Khurai': [24.043468, 78.330353],
 'Upleta': [21.741323, 70.28112],
 'Saharanpur': [29.959694, 77.549057],
 'Ghaziabad': [28.667856, 77.449791],
 'Sambalpur': [21.468185, 83.975403],
 'Saharsa': [25.883495, 86.600624],
 'Gohpur': [26.880077, 93.60833],
 'Narsapur': [16.432983, 81.696617],
 'Malappuram': [11.072035, 76.074005],
 'Gauribidanur': [13.611159, 77.51696],
 'Dibrugarh': [27.472834, 94.911964],
 'Gangtok': [27.338936, 88.606506],
 'Balotra': [25.83087, 72.24012],
 'Narsampet': [17.928122, 79.894531],
 'Kozhikode': [11.258753, 75.780411],
 'Hindaun City': [26.731142, 77.033752],
 'Kumbakonam': [10.959815, 79.380898],
 'Satna': [24.579716, 80.832176],
 'Chhindwara': [22.057163, 78.938202],
 'Ongole': [15.505723, 80.049919],
 'Padrauna': [26.898392, 83.979713],
 'Akola': [20.707228, 77.00296],
 'Kalwan': [20.492281, 74.026009],
 'Sircilla': [18.38662, 78.802246],
 'Rohtak': [28.895515, 76.606613],
 'Anandapur': [21.2155, 86.1222],
 'DCKAP Technologies': [13.090994, 80.224998],
 'Garhshankar': [31.213682, 76.144012],
 'Jagtial': [18.794905, 78.912872],
 'Dausa': [26.894478, 76.334824],
 'Muradnagar': [28.771646, 77.507561],
 'Srivilliputhur': [9.512137, 77.634087],
 'Paramakudi': [9.548395, 78.591637],
 'Barabanki': [26.937834, 81.188324],
 'Secunderabad': [17.43993, 78.498276],
 'Mirzapur': [25.133699, 82.56443],
 'Virudhachalam': [11.519588, 79.325157],
 'Jhunjhunu': [28.128876, 75.399506],
 'Kottayam': [9.591441, 76.522171],
 'Shivpuri': [25.42412, 77.65799],
 'Amalner': [21.04241, 75.063873],
 'Durgapur': [23.53344, 87.32193],
 'Gola Gokrannath Kheri': [28.080379, 80.466644],
 'Dharwad': [15.460252, 75.010284],
 'Vapi': [20.385181, 72.911453],
 'Firozabad': [27.159101, 78.39576],
 'Vasco da Gama': [15.386033, 73.84404],
 'Asansol': [23.673944, 86.952393],
 'Tenkasi': [8.959021, 77.312935],
 'Srikakulam': [18.296974, 83.896782],
 'Hassan': [13.009711, 76.102898],
 'Churu': [28.29229, 74.966583],
 'Bulandshahr': [28.406963, 77.849831],
 'Hinganghat': [20.550573, 78.841141],
 'Lakhisarai': [25.168901, 86.092987],
 'Mahoba': [25.292509, 79.872665],
 'Sirsa': [29.536535, 75.025505],
 'Ranaghat': [23.174049, 88.563934],
 'Udgir': [18.3934, 77.113144],
 'Ratlam': [23.334169, 75.037636],
 'Nimbahera': [24.620001, 74.68],
 'Robertsganj': [24.685001, 83.068352],
 'Khandwa': [21.825733, 76.35257],
 'Rajendra Nagar': [20.450474, 85.905533],
 'Mehsana': [23.586761, 72.369949],
 'Fatehabad': [29.511778, 75.455215],
 'Raiganj': [25.61853, 88.125587],
 'Junnar': [19.20928, 73.872589],
 'Unchahar': [25.9, 81.300003],
 'Banka': [24.885483, 86.920731],
 'Jamner': [20.809378, 75.778923],
 'Khamgaon': [20.711622, 76.566132],
 'Mehkar': [20.149822, 76.574036],
 'Rewa': [24.530727, 81.29911],
 'Karnal': [29.685629, 76.990547],
 'Ichalkaranji': [16.709002, 74.456078],
 'Bihar Sharif': [25.19935, 85.517609],
 'Cooch Behar': [26.323921, 89.451088],
 'Imphal': [24.813967, 93.950279],
 'Viramgam': [23.125837, 72.057442],
 'Anantnag': [33.729729, 75.14978],
 'Panjim': [15.496777, 73.827827],
 'Anand': [22.554029, 72.948936],
 'Manmad': [20.251549, 74.438354],
 'Daryapur': [20.92207, 77.326698],
 'Arvi': [20.634085, 79.140388],
 'Paratwada': [21.27306, 77.523422],
 'Bardoli': [21.124857, 73.11261],
 'Bodhgaya': [24.696135, 84.986954],
 'Chabua': [27.48, 95.18],
 'Jamkhandi': [16.504227, 75.290932],
 'Burdwan': [23.232513, 87.863419],
 'Tumakuru': [13.338263, 77.10141],
 'Nayagarh': [20.127028, 85.105057],
 'Sivasagar': [26.98, 94.629997],
 'Wani': [20.067541, 78.958054],
 'Karwar': [14.801439, 74.124069],
 'Kapurthala': [31.379999, 75.379997],
 'Ganderbal': [34.216496, 74.771942],
 'Guna': [24.65, 77.32],
 'Khanna': [30.702877, 76.220222],
 'Mahbubnagar': [16.737509, 78.008125],
 'Nalgonda': [17.05662, 79.268333],
 'Theni': [10.010366, 77.476814],
 'Eluru': [16.703285, 81.100388],
 'Pusad': [19.912676, 77.56691],
 'Shamshabad': [17.25116, 78.437737],
 'Aurangabad': [24.75, 84.370003],
 'Bankura': [23.233774, 87.08725],
 'Renukoot': [24.220198, 83.035957],
 'Anakapalle': [17.686001, 83.008781],
 'Sikandrabad': [28.452393, 77.697205],
 'Saidabad': [28.526592, 77.290169],
 'Yelahanka': [13.094454, 77.586014],
 'Pithoragarh': [29.582861, 80.218185],
 'Vidisha': [23.530001, 77.82],
 'Warora': [20.23, 79.0],
 'Jhagadia': [21.719477, 73.151009],
 'Morbi': [22.811989, 70.823616],
 'Bhopal': [23.259933, 77.412613],
 'Agartala': [23.829321, 91.277847],
 'Bhandara': [21.17, 79.650002],
 'Baramati': [18.150663, 74.576782],
 'Makrana': [27.029617, 74.716324],
 'Panipat': [29.398928, 76.977081],
 'Sehore': [23.198437, 77.095184],
 'Madanapalle': [13.55, 78.5],
 'Madhopur': [32.365288, 75.59729],
 'Bhuj': [23.242001, 69.666931],
 'Shahjahanpur': [27.883743, 79.912247],
 'Raebareli': [26.230299, 81.240891],
 'Dhanbad': [23.795399, 86.42704],
 'Hasanpur': [28.719999, 78.279999],
 'Jabalpur': [23.16877, 79.931847],
 'Shahada': [21.546185, 74.470009],
 'Mysuru': [12.29581, 76.639381],
 'Panskura': [22.395582, 87.741882],
 'Bijnor': [29.372442, 78.135849],
 'Rajpura': [30.483997, 76.593948],
 'Pipariya': [22.762886, 78.352478],
 'Auraiya': [26.469999, 79.519997],
 'Bahadurgarh': [28.68, 76.919998],
 'Avinashi': [11.191447, 77.268883],
 'Thanesar': [29.962072, 76.817825],
 'Malout': [30.191515, 74.497864],
 'Pilkhuwa': [28.711969, 77.654457],
 'Jind': [29.316668, 76.316666],
 'Santipur': [23.255541, 88.437363],
 'Sathankulam': [8.444186, 77.912804],
 'Amravati': [20.937424, 77.779549],
 'Hathras': [27.6, 78.050003],
 'Arcot': [12.904441, 79.31916],
 'Khurja': [28.252222, 77.851784],
 'Bhuvanagiri': [17.518345, 78.885956],
 'Kavesar': [19.262466, 72.974106],
 'Rajgangpur': [22.205206, 84.584251],
 'Fazilka': [30.403648, 74.027962],
 'Buldana': [20.536846, 76.18087],
 'Amroha': [28.902397, 78.465042],
 'Nedumangad': [8.603333, 77.002777],
 'Sawaimadhopur': [25.983334, 76.366669],
 'Akot': [21.1, 77.059998],
 'Thane West': [19.226662, 72.983833],
 'Balasore': [21.494978, 86.942657],
 'Doddaballapura': [13.298757, 77.540817],
 'Mettupalayam': [11.289087, 76.940971],
 'Mallaguri': [26.743464, 88.416252],
 'Sitapur': [27.57509, 80.663765],
 'Jaora': [23.644524, 75.131035],
 'Dhuri': [30.370001, 75.870003],
 'Sitalkuchi': [26.169069, 89.180122],
 'Dombivli': [19.2094, 73.093948],
 'Pathankot': [32.265942, 75.646873],
 'Thiruvananthapuram': [8.524139, 76.936638],
 'Beawar': [26.100735, 74.319077],
 'Ajmer': [26.449896, 74.639915],
 'Jagraon': [30.780001, 75.480003],
 'Budaun': [28.033709, 79.120544],
 'Pandharpur': [17.674553, 75.323723],
 'Damoh': [23.839054, 79.441025],
 'Chintamani': [13.401969, 78.055138],
 'Baramulla': [34.202148, 74.348259],
 'Murshidabad': [24.175903, 88.280182],
 'Bheemili': [17.887518, 83.445679],
 'Patiala': [30.34, 76.379997],
 'Kochi': [9.939093, 76.270523],
 'Muzaffarpur': [26.121473, 85.368752],
 'Hoskote': [13.07, 77.800003],
 'Kalol': [22.606001, 73.462997],
 'Kakinada': [16.989065, 82.247467],
 'Panchkula': [30.695202, 76.854172],
 'Greater Noida': [28.474388, 77.50399],
 'Barwani': [22.030001, 74.900002],
 'Hubballi': [15.364708, 75.123955],
 'Udamanda': [30.321989, 79.195412],
 'Deoghar': [24.482775, 86.695175],
 'Indore Bhopal Road': [23.200001, 77.080002],
 'North Paravoor': [10.140487, 76.23053],
 'Hazira': [21.116226, 72.651802],
 'Bargadi Magath': [26.983795, 80.92907],
 'Tarabai Park': [16.713512, 74.248749],
 'Kanchrapara': [22.94923, 88.420921],
 'Puttur': [12.768676, 75.207062],
 'Ambala City': [30.37818, 76.776695],
 'Faridkot': [30.678186, 74.739555],
 'Osmanabad': [18.186066, 76.041939],
 'Raichur': [16.200829, 77.362289],
 'Chitradurga': [14.23, 76.400002],
 'Mysore division': [12.311827, 76.652985],
 'Lalitpur': [24.68786, 78.412018],
 'Agar Malwa': [23.710348, 76.010178],
 'Shamli': [29.448055, 77.31028],
 'Indirapuram': [28.641485, 77.371384],
 'Mansoorabad': [17.351372, 78.565407],
 'Tirupati': [13.629065, 79.424446],
 'Sojat': [25.923853, 73.665131],
 'Palghar': [19.693558, 72.765518],
 'Gondia': [21.459801, 80.195],
 'Bhadravati': [13.84, 75.702003],
 'Balrampur': [27.429707, 82.176804],
 'Kawardha': [22.008951, 81.224342],
 'Nellore': [14.442599, 79.986458],
 'Karimnagar': [18.438555, 79.128838],
 'Lonavala': [18.74806, 73.407219],
 'Sujangarh': [27.696903, 74.452972],
 'Kothrud': [18.50989, 73.807182],
 'Haridwar': [29.94569, 78.164246],
 'Vashi': [19.077065, 72.998993],
 'Moradabad': [28.841064, 78.795319],
 'Chirimiri': [23.18795, 82.354164],
 'Gorakhpur': [26.765844, 83.364944],
 'Siwan': [26.219622, 84.356659],
 'Adambakkam': [12.981241, 80.20945],
 'Chhapra': [25.779566, 84.749886],
 'Surat': [21.17024, 72.831062],
 'Sheopur': [25.67, 76.699997],
 'Phagwara': [31.22402, 75.770798],
 'Sendhwa': [21.681898, 75.094269],
 'Daman': [20.397373, 72.832802],
 'Narnaul': [28.065796, 76.101479],
 'Deesa': [24.258503, 72.190674],
 'Nabha': [30.373674, 76.145187],
 'Zirakpur': [30.642803, 76.816902],
 'Chachiyawas': [26.570478, 74.674438],
 'Kila': [28.23801, 77.854912],
 'Raigarh': [21.9, 83.400002],
 'Haldia': [22.066673, 88.069809],
 'Bidar': [17.92, 77.519722],
 'Nipani': [16.4, 74.383301],
 'Dhamtari': [20.709999, 81.550003],
 'Bilaspur': [22.078642, 82.152328],
 'Jagdalpur': [19.07, 82.029999],
 'Bhavani': [11.452434, 77.693253],
 'Jaisalmer': [26.911661, 70.922928],
 'Puthoor': [9.04, 76.709999],
 'Berhampore': [24.098827, 88.267929],
 'Mira Bhayandar': [19.295233, 72.854393],
 'Khadki': [18.569937, 73.850639],
 'Barasat': [22.726156, 88.47496],
 'Sonepat': [28.928574, 77.091492],
 'Nawapara Nagar': [20.973513, 81.857948],
 'Bally': [22.655149, 88.340553],
 'Ghansoli': [19.125362, 72.999199],
 'Aligarh': [27.900383, 78.072281],
 'Kadi': [23.29785, 72.331001],
 'Machilipatnam': [16.17, 81.129997],
 'Jamui': [24.92, 86.220001],
 'Amalsad': [20.817595, 72.954918],
 'Pathardi Phata': [19.946922, 73.765434],
 'Nadiad': [22.700001, 72.870003],
 'Chikhli': [20.349098, 76.265289],
 'Krishnagiri': [12.518611, 78.213737],
 'Cochin': [9.931233, 76.267303],
 'Katni': [23.833717, 80.420609],
 'Ooty': [11.41, 76.699997],
 'Hajipur': [25.68, 85.220001],
 'Malad East': [19.180874, 72.857452],
 'Davangere': [14.470586, 75.914154],
 'Karjat': [18.920902, 73.323097],
 'Kattuvilai': [8.158933, 77.336746],
 'Kalamassery': [10.051969, 76.315773],
 'Bijapur': [16.827545, 75.725327],
 'Kajamalai': [10.775632, 78.69236],
 'Malda': [25.010841, 88.141098],
 'Dadabari': [25.155169, 75.827248],
 'Anantapur': [14.685564, 77.595406],
 'Nawada': [24.879999, 85.529999],
 'Bikaner': [28.027138, 73.302155],
 'Falta': [22.296404, 88.120995],
 'Puri': [19.817743, 85.828629],
 'Vikramasingapuram': [8.699142, 77.411041],
 'Faizabad': [26.77, 82.150002],
 'Kurukshetra': [29.969513, 76.878281],
 'Jalukbari': [26.144358, 91.642067],
 'Jaipur': [26.92207, 75.778885],
 'Kamptee': [21.207769, 79.184052],
 'Dharavi': [19.044218, 72.85598],
 'Chomu': [27.17, 75.720001],
 'Mudhol': [16.333309, 75.28582],
 'Jamnagar': [22.470701, 70.057732],
 'Hojai': [26.006807, 92.866402],
 'Agra': [27.17667, 78.008072],
 'Morena': [26.5, 78.0],
 'Kalyan': [19.242439, 73.120193],
 'Dadri': [28.54619, 77.556213],
 'Hansi': [29.102377, 75.965996],
 'Bundi': [25.430513, 75.649902],
 'Ujjain': [23.1793, 75.784912],
 'Ejipura': [12.942621, 77.627121],
 'Noida': [28.535517, 77.391029],
 'Bongaon': [23.07, 88.82],
 'Dalkhola': [25.874945, 87.845551],
 'Katihar': [25.533508, 87.583748],
 'Bhankrota': [26.87063, 75.693398],
 'Andheri': [19.116625, 72.862358],
 'Tuticorin': [8.764166, 78.134834],
 'Pollachi': [10.657274, 77.010666],
 'Thane': [19.21833, 72.978088],
 'North Lakhimpur': [27.236317, 94.09584],
 'Ennore': [13.206524, 80.327225],
 'Tirupattur': [12.498351, 78.560196],
 'Motihari': [26.65, 84.916664],
 'Tikamgarh': [24.756807, 78.839264],
 'Borivali': [19.228825, 72.854118],
 'Datia': [25.665325, 78.460938],
 'Jorhat': [26.757874, 94.209824],
 'Cuttack': [20.462521, 85.882988],
 'Vadodara': [22.310696, 73.192635],
 'Sikar': [27.615406, 75.125885],
 'Dewas': [22.962267, 76.050797],
 'Malegaon': [20.560797, 74.52507],
 'Bapatla': [15.896622, 80.460434],
 'Hanumangarh': [29.625996, 74.287491],
 'Devlali': [19.906166, 73.824265],
 'Sunam': [30.129999, 75.800003],
 'Vastral': [22.99988, 72.660614],
 'Bareilly': [28.375694, 79.435959],
 'Nanded': [19.169815, 77.319717],
 'Umred': [20.861378, 79.317856],
 'Hubli': [15.371598, 75.101166],
 'Barrackpore': [22.767427, 88.388344],
 'Shajapur': [23.4333, 76.266701],
 'Jalandhar': [31.326015, 75.57618],
 'Gandhinagar': [23.23756, 72.647781],
 'Jalaun': [26.140644, 79.355621],
 'Rupnagar': [30.975254, 76.527328],
 'Adyar': [13.003387, 80.255043],
 'Guwahati': [26.148043, 91.731377],
 'Bhiwani': [28.799046, 76.133514],
 'Koyambedu': [13.07279, 80.176163],
 'Tiruchengode': [11.378476, 77.894493],
 'Palanpur': [24.179331, 72.426682],
 'Nagercoil': [8.17445, 77.432159],
 'Coimbatore': [11.004556, 76.961632],
 'Kanpur': [26.449923, 80.331871],
 'Bellampalli': [19.071556, 79.491173],
 'Sangli': [16.867634, 74.570389],
 'Kadapa': [14.477234, 78.804932],
 'Palwan': [18.970057, 75.72065],
 'Kurnool': [15.834536, 78.029366],
 'Visakhapatnam': [17.686815, 83.218483],
 'Tirunelveli': [8.741222, 77.694626],
 'Kharar': [30.751278, 76.637192],
 'Vadapalani': [13.052192, 80.212051],
 'Bhadrak': [21.051182, 86.485748],
 'Madha': [18.033587, 75.522079],
 'Bhubaneshwar': [20.296059, 85.824539],
 'Ahmednagar': [19.101053, 74.740677],
 'Adilabad': [19.6667, 78.533302],
 'Alambagh': [26.805687, 80.914993],
 'Sangamner': [19.581099, 74.205009],
 'Bettiah': [26.799999, 84.5],
 'Thanjavur': [10.786999, 79.137825],
 'Amritsar': [31.63398, 74.872261],
 'Solapur': [17.65992, 75.906387],
 'Ghatampur': [26.164066, 80.17067],
 'Islampur': [26.266701, 88.199997],
 'Washim': [20.1, 77.150002],
 'Pudukkottai': [10.379663, 78.820847],
 'Malvan': [16.0667, 73.466698],
 'Chittoor': [13.217096, 79.100677],
 'Dehradun': [30.316496, 78.032188],
 'Kargil': [34.556335, 76.132507],
 'Warangal': [18.000055, 79.588165],
 'Kolhapur': [16.691307, 74.244865],
 'Nagaur': [27.207006, 73.742294],
 'Pithapuram': [17.112427, 82.252899],
 'Kannur': [11.874477, 75.370369],
 'Darbhanga': [26.152973, 85.901413],
 'Vellore': [12.934968, 79.146881],
 'Bicholim': [15.601875, 73.954636],
 'Tirur': [10.914627, 75.922096],
 'Rajsamand': [25.0667, 73.883301],
 'Shrirampur': [19.620296, 74.654655],
 'Jodhpur': [26.263863, 73.008957],
 'Indore': [22.719568, 75.857727],
 'Sathy': [11.504776, 77.238396],
 'Kothamangalam': [10.065206, 76.629128],
 'Srinagar': [30.221399, 78.780045],
 'Dimapur': [25.906267, 93.727592],
 'Kannauj': [27.055189, 79.918083],
 'New Delhi': [28.6448, 77.216721],
 'Latur': [18.407957, 76.576767],
 'Shimla': [31.104605, 77.173424],
 'Dungargarh': [28.095606, 74.011116],
 'Varanasi': [25.321684, 82.987289],
 'Nashik': [19.997454, 73.789803],
 'Mandla': [22.597921, 80.371384],
 'Udumalaipettai': [10.5833, 77.25],
 'Sriperumbudur': [12.97378, 79.94133],
 'Tiruppur': [11.110695, 77.348045],
 'Chennai': [13.067439, 80.237617],
 'Parbhani': [19.25779, 76.773743],
 'Rampur': [28.802958, 79.025444],
 'Harda': [22.356426, 77.102737],
 'Dharampur': [20.54014, 73.179214],
 'Salem': [11.664325, 78.146011],
 'Chengalpattu': [12.693933, 79.975662],
 'Mathura': [27.492413, 77.673676],
 'Namakkal': [11.229592, 78.171158],
 'Mangrulpir': [20.316135, 77.341171],
 'Vijayawada': [16.515099, 80.632095],
 'Basti': [26.822845, 82.763443],
 'Burhanpur': [21.307373, 76.230415],
 'Virudhunagar': [9.587209, 77.951431],
 'Madurai': [9.939093, 78.121719],
 'Bhagalpur': [25.253391, 86.989059],
 'Tambaram': [12.922915, 80.127457],
 'Sirhind': [30.64382, 76.394119],
 'Nawalgarh': [27.862146, 75.254288],
 'Ajitgarh': [30.7076, 76.715126],
 'Purulia': [23.342257, 86.362839],
 'Aland': [17.570721, 76.569733],
 'Kukatpally': [17.50001, 78.401527],
 'Kopargaon': [19.883301, 74.483299],
 'Gwalior': [26.218287, 78.182831],
 'Tenali': [16.236719, 80.647476],
 'Manwath': [19.299999, 76.5],
 'Satara': [17.691401, 74.000938],
 'Kishangarh': [26.588528, 74.872513],
 'Kosumb': [17.11458, 73.591805],
 'Partur': [19.595484, 76.211067],
 'Modinagar': [28.838299, 77.581055],
 'Mandi Gobindgarh': [30.682802, 76.294212],
 'Chimur': [20.501314, 79.380173],
 'Indapur': [18.11714, 75.023918],
 'Katraj': [18.445089, 73.86898],
 'Dhayari': [18.441833, 73.812157],
 'Saswad': [18.344749, 74.031715],
 'Pimpri': [18.626076, 73.812157],
 'Nangal Dam': [31.385296, 76.374893],
 'Beed': [18.992012, 75.776138],
 'Parli Vaijnath': [18.871204, 76.535568],
 'Villupuram': [11.94596, 79.497414],
 'Sivakasi': [9.453126, 77.807236],
 'Jammu': [32.732998, 74.864273],
 'Thrissur': [10.530345, 76.214729],
 'Athani': [16.724083, 75.063843],
 'Vizag': [17.690474, 83.231049],
 'Guntur': [16.314209, 80.435028],
 'Bhadgaon': [20.666668, 75.23333],
 'Jaipatna': [19.482777, 82.806702],
 'Banda': [25.492249, 80.336151],
 'Godda': [24.827778, 87.211388],
 'Rajahmundry': [17.004393, 81.783325],
 'Kadiri': [14.117532, 78.15509],
 'Ahmedabad': [23.033863, 72.585022],
 'Mavdi': [22.259392, 70.777184],
 'Dhenkanal': [20.660671, 85.596954],
 'Hyderabad': [17.38714, 78.491684],
 'Patna': [25.612677, 85.158875],
 'Chandigarh': [30.741482, 76.768066],
 'Melur': [10.030623, 78.34021],
 'Hisar': [29.151861, 75.721123],
 'Junagadh': [21.515471, 70.456444],
 'Halol': [22.496063, 73.475449],
 'Srikalahasti': [13.75269, 79.703758],
 'Dankuni': [22.671679, 88.300209],
 'Pilibhit': [28.627926, 79.804176],
 'Buti Bori': [20.927933, 79.004143],
 'Jalna': [19.846811, 75.890633],
 'Dhampur': [29.257648, 78.500061],
 'Udaipur': [24.57127, 73.691544],
 'Ferozepur': [30.923433, 74.610214],
 'Gannavaram': [16.538591, 80.798218],
 'Virar': [19.465622, 72.806099],
 'Rajkot': [22.308155, 70.800705],
 'Tezpur': [26.651218, 92.783813],
 'Thalassery': [11.74805, 75.48938],
 'Dhule': [20.903118, 74.774986],
 'Rishikesh': [30.08716, 78.268112],
 'Bazpur': [29.162411, 79.152168],
 'Gonda': [27.133875, 81.96199],
 'Sukdal': [23.356615, 87.656837],
 'Nagpur': [21.146633, 79.08886],
 'Julana': [29.121574, 76.39772],
 'Konavani Palem': [17.417376, 82.741653],
 'Khalapur': [18.834003, 73.288712],
 'Sedam': [17.179991, 77.275368],
 'Gurgaon': [28.457523, 77.026344],
 'Pune': [18.516726, 73.856255],
 'Ambikapur': [23.116444, 83.196121],
 'Savner': [21.385311, 78.92186],
 'Erode': [11.342423, 77.728165],
 'Jamshedpur': [22.805618, 86.20311],
 'Varthur': [12.938482, 77.747711],
 'Karad': [17.286501, 74.181427],
 'Bangalore': [12.972442, 77.580643],
 'Egra': [21.900526, 87.538078],
 'Thirunallar city': [10.930677, 79.787643],
 'Gaya': [24.78001, 84.981827],
 'Unjha City': [23.804337, 72.393677],
 'Nalasopara': [19.432278, 72.7743],
 'Kolkata': [22.572645, 88.363892],
 'Amaravati': [20.9320, 77.7523],
 'Itanagar': [27.0844, 93.6053],
 'Dispur': [26.1433, 91.7898],
 'Panaji': [15.4909, 73.8278],
 'Puducherry': [11.9416, 79.8083],
                }

In [ ]:
dist_l = []
for i in l2:
    dist_l += [lat_long_dict[capitals_dict[i]]]

In [ ]:
def make_x0(l2,statewise_pop):
    x = np.zeros((len(l2),len(l2)))
    for j in range(len(l2)):
        row = []
        for i in range(len(l2)):
            if i==j:
                row += [0]
            else:
                row += [statewise_pop[l2[i]]]
        row = np.array(row)
        row = row/row.sum()
        row[j] = 9
        row *= 0.1*statewise_pop[l2[j]]
        x[j:] = row
    return x

from math import radians, cos, sin, asin, sqrt

def distance(lat1, lat2, lon1, lon2):
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
       
    # Haversine formula  
    dlon = np.abs(lon2 - lon1)  
    dlat = np.abs(lat2 - lat1)
    a = sin(dlat / 2)*2 + cos(lat1) * cos(lat2) * sin(dlon / 2)*2
    c = 2 * asin(sqrt(a))  
     
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
    
    ret = c*r
    if ret==0:
        ret = 100
    return ret

def make_x01(dist_l,statewise_pop):
    x = np.zeros((len(l2),len(l2)))
    for i in range(len(l2)):
        lat1, lon1 = dist_l[i]
        for j in range(len(l2)):
            lat2, lon2 = dist_l[j]
            if i!=j:
                x[i,j] = 1/distance(lat1, lat2, lon1, lon2)
    x = x/x.sum(axis=1).reshape(-1,1)
    for i in range(len(x)):
        x[i,i] = 9
        x[i,:] *= 0.1*statewise_pop[l2[i]] 
    return x

x01 = make_x0(l2,statewise_pop) - make_x0(l2,statewise_pop).diagonal()
x02 = make_x01(dist_l,statewise_pop) - make_x01(dist_l,statewise_pop).diagonal()

x03 = x01*x02
x03 = x03/x03.sum(axis=1).reshape(-1,1)/10

x03 = np.eye(len(x03))*0.9+x03

population = [statewise_pop[l2[i]] for i in range(len(l2))]

X0  = x03* np.array(population).reshape(-1,1)


In [ ]:
def make_Y0(Y0,big_df,l2):
    Y0 *= 0
    VAL = big_df.loc[40]
    for i in range(len(l2)):
        name = l2[i]+'_I_cumsum'
        Y0[i,i] = VAL[name]
    return Y0

Y0 = make_Y0(Y0,big_df,l2)
W0 = Y0

## Transitions

In [ ]:
def f(t,y,y_,B=0,sign=1,dim=1):
    n = dim*dim
    X,W,Y,Z,D = [y[i*n:i*n+n].reshape(dim,dim) for i in range(5)]
    N = X+Y+W+Z
    ret =  sign*B*X*(Y.sum(axis=1)/N.sum(axis=1)).reshape(-1,1)
    return ret.ravel()

def g(t,y,y_,sig=0,sign=1,dim=1):
    n = dim*dim
    X,W,Y,Z,D = [y[i*n:i*n+n].reshape(dim,dim) for i in range(5)]
    ret = sign*sig*W
    return ret.ravel()

def h(t,y,y_,gam=0,rho=0,sign=1,dim=1):
    n = dim*dim
    X,W,Y,Z,D = [y[i*n:i*n+n].reshape(dim,dim) for i in range(5)]
    ret = sign*gam*Y*(1-rho)
    return ret.ravel()

def q(t,y,y_,rho=0.01,sign=1,dim=1):
    n = dim*dim
    X,W,Y,Z,D = [y[i*n:i*n+n].reshape(dim,dim) for i in range(5)]
    ret = sign*rho*Y
    return ret.ravel()

def mobility(t,y,y_,which='X',l=0,r_f=0,C=0,sign=1,dim=1,S0=0):
    n = dim*dim
    temp_dict = {}
    for k,v in zip(['X','W','Y','Z','D'],[y[i*n:i*n+n].reshape(dim,dim) for i in range(5)]):
        temp_dict[k] = v
    x = temp_dict[which]
    np.random.seed(S0+int(t))
    r = r_f()
    ret = sign*C* ((-l.sum(axis=0)*(np.eye(dim)*np.diag(x)) + l*np.diag(x)) +
                  (+(r*x).sum(axis=0)*np.eye(dim) - r*x))
    return ret.ravel()     


## Model

In [ ]:
B = np.array(R0).reshape(-1,1)/3
sig = 1/7
gam = 1/3
rho = 0

l = np.array([[0.]])

S0 = 0

def R_f(dim=1):
    mat = 0.1*np.random.random((dim,dim))
    return (mat-np.eye(dim)*np.diag(mat))

r_f = partial(R_f,dim=dim)
C = 0.5

model = Multi_SEIR(dim,[X0,W0,Y0,Z0,D0])  

model.add('X_d', [partial(f,B=B,sign=-1),
              partial(mobility,r_f=r_f,l=l,C=C,which='X',S0=S0),
                ])

model.add('W_d', [partial(f,B=B,sign=1),
              partial(g,sig=sig,sign=-1),
              partial(mobility,r_f=r_f,l=l,C=C,which='W',S0=S0),])


model.add('Y_d', [partial(g,sig=sig,sign=1),
                partial(h,gam=gam,rho=rho,sign=-1),
                partial(q,rho=rho,sign=-1),
              partial(mobility,r_f=r_f,l=l,C=C,which='Y',S0=S0),
                ])

model.add('Z_d', [partial(h,gam=gam,rho=rho,sign=1),
              partial(mobility,r_f=r_f,l=l,C=C,which='Z',S0=S0),
             ])

model.add('D_d', [partial(q,rho=rho,sign=1)])


days = 200
model.solve(range(days))


## Plots

In [ ]:
from cycler import cycler

def rename(name):
    l = [name[0]]
    for i in name[1:]:
        if i!=i.lower():
            l += [' ']
        l += [i]
    return ''.join(l)

labels = ['Susceptible','Exposed','Infected','Removed','']

y = model.y

n = model.n_population*model.n_population
y = y.reshape(days,5,model.n_population,model.n_population).sum(axis=-1)

N_ = y[:,:4,:].sum(axis=1)
N = y[:,:,:].sum(axis=1)

In [ ]:
set_font(16,plt)

for i in range(model.n_population):
    fig, ax = plt.subplots(1,1,dpi=400)
    # plot all 
    ax.plot(N[:,i]/1e6,'--',label='Initial Pop.')
    for j in range(1,4):
        y3 = y[:,j,i].astype(np.int)
        y4 = []
        for k in y3:
            if k>0.5:
                y4 += [k/1e6]
            else:
                y4 += [np.nan]
                    
        ax.plot(y4,label='{}'.format(labels[j]),) 

    y2 = big_df[l2[i]+'_I_cumsum'].loc[range(40,67)].values
    ax.plot(range(len(y2)),y2/1e6,'x',label='Observed',c='k',mew=3)

    ax.set_title("{}".format("{} (I$_0$ = {})".format(rename(l2[i]),y2[0])))
    ax.set_yscale('log')
    plt.xlabel('Time (day)')
    plt.ylabel('Population (million)')
    ax.legend()
    fig.show()
    